## Caching Return Values

<span style="color:orange">**Note:**</span> 
Use the `Least Recently Used (LRU) cache` available as `@functools.lru_cache`instead of writing your own cache decorator.

### Example 1:

- In this example, the `add_cache` function is a class method decorator that takes a method as its argument, adds caching functionality to it by storing the results of each method call in a dictionary, and returns a wrapper function that either returns the cached result or calculates the result and caches it.

- The `Calculator` class has a single method, `add`, which simply adds two numbers together and prints a message to indicate that it's performing the calculation. 

- The `@add_cache decorator` is applied to this method, which means that the results of each call to add will be cached.

- Then two different instances of `Calculator` is created, `calculator1` and `calculator2`.
    - The first time `add` method is called on `calculator1`, the method performs the calculation and caches the result. 

    - The second time it's called, the method simply returns the cached result without performing the calculation again. 
    
    - This demonstrates how the caching functionality works.

In [1]:
import functools

def add_cache(method):
    """
    This decorator takes a class method as an argument and returns a wrapped function that adds caching
    functionality to the method. The wrapped function caches the results of previous method calls, and 
    returns the cached result if the same arguments are used again. This can improve performance by 
    avoiding unnecessary repeated computation.

    The cache is stored in a dictionary within the decorator function, and is only accessible within the
    scope of the decorator. The decorator returns the wrapped function, which can be used in place of the 
    original method.

    Args:
        method: The class method to be wrapped with caching functionality.

    Returns:
        The wrapped function that adds caching functionality to the original method.
    """
    cache = {}  
    
    @functools.wraps(method)
    def wrapper_add_cache(self, *args):
        if args in cache: # check if the arguments are in the cache
            print(f"Using cached result for {method.__name__}{args}.")
            return cache[args]
        result = method(self, *args) # if the args are not in the cache, call the method
        cache[args] = result # store the results from the method call
        print(f"Caching result for {method.__name__}{args}.")
        return result 
    return wrapper_add_cache


class Calculator:
    @add_cache
    def add(self, x, y):
        print(f"Calculating {x} + {y}.")
        return x + y

calculator1 = Calculator() # define a new instance of the Calculator class
result = calculator1.add(10, 10)
print(result)
result = calculator1.add(2, 2)
print(result)

calculator2 = Calculator() 
result = calculator2.add(10, 10) # this is already cached
print(result)

Calculating 10 + 10.
Caching result for add(10, 10).
20
Calculating 2 + 2.
Caching result for add(2, 2).
4
Using cached result for add(10, 10).
20


### Example 2

In [2]:
from decorators import count_calls

def cache(func):
    """
    cache(func)
    -----------

    A decorator function that caches the result of the given function 
    for future calls with the same arguments.

    Parameters
    ----------
    func : callable
    The function to be decorated.

    Returns
    -------
    wrapper_cache : callable
    A new function that wraps around the given function and caches its results 
    for future calls with the same arguments.

    Example
    -------
    Consider the following example:

    @cache
    def fibonacci(n):
        if n <= 1:
            return n
        return fibonacci(n-1) + fibonacci(n-2)

    print(fibonacci(10))  # the result is computed and cached
    print(fibonacci(10))  # the cached result is returned

    In this example, the `fibonacci` function is decorated with `cache`, 
    which allows the function to be called with the same argument multiple times 
    without having to recompute the result each time. 
    The first call to `fibonacci(10)` computes the result and caches it,
    while the second call returns the cached result without recomputing it, 
    resulting in a faster execution time.
    """
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = dict()
    return wrapper_cache

@cache
@count_calls
def fibonacci(num):
    if num < 2:
        return num 
    return fibonacci(num - 1) + fibonacci(num - 2)    

In [3]:
fibonacci(10)

Call 1 of 'fibonacci'
Call 2 of 'fibonacci'
Call 3 of 'fibonacci'
Call 4 of 'fibonacci'
Call 5 of 'fibonacci'
Call 6 of 'fibonacci'
Call 7 of 'fibonacci'
Call 8 of 'fibonacci'
Call 9 of 'fibonacci'
Call 10 of 'fibonacci'
Call 11 of 'fibonacci'


55

In [4]:
fibonacci(8)

21

In [5]:
fibonacci(13)

Call 12 of 'fibonacci'
Call 13 of 'fibonacci'
Call 14 of 'fibonacci'


233

## Example 3

- The `maxsize` parameter specifies how many recent calls are cached - default 128.

- The `.cache_info()` method shows how the cache performs - great for tuning the logic. 

- The artificially small maxsize in the example shows the side effect of elements being removed from the cache.

In [6]:
@functools.lru_cache(maxsize=4)
def fibonacci(num):
    print(f"Calculating fibonacci({num})")
    if num < 2:
        return num 
    return fibonacci(num - 1) + fibonacci(num - 2)

In [7]:
fibonacci(10)

Calculating fibonacci(10)
Calculating fibonacci(9)
Calculating fibonacci(8)
Calculating fibonacci(7)
Calculating fibonacci(6)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)


55

In [8]:
fibonacci(8)

21

In [9]:
fibonacci(5)

Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)


5

In [10]:
fibonacci(8)

Calculating fibonacci(8)
Calculating fibonacci(7)
Calculating fibonacci(6)


21

In [11]:
fibonacci(5)

5

In [12]:
fibonacci.cache_info()

CacheInfo(hits=17, misses=20, maxsize=4, currsize=4)